# 第8回講義 宿題

## 課題. Theanoを用いて, CIFAR-10を畳み込みニューラルネットワーク(CNN)で学習せよ

### 注意

- homework関数を完成させて提出してください
    - 訓練データはtrain_X, train_y, テストデータはtest_Xで与えられます
    - train_Xとtrain_yをtrain_X, train_yとvalid_X, valid_yに分けるなどしてモデルを学習させてください
    - test_Xに対して予想ラベルpred_yを作り, homework関数の戻り値としてください\
- pred_yのtest_yに対する精度(F値)で評価します
- 全体の実行時間がiLect上で60分を超えないようにしてください
- homework関数の外には何も書かないでください

次のような内容のコードが**事前**に実行されます

```python
from __future__ import division
from collections import OrderedDict
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.cross_validation import train_test_split
from theano.tensor.nnet import conv2d
from theano.tensor.signal import pool
from theano.tensor.shared_randomstreams import RandomStreams

import cPickle
import numpy as np
import theano
import theano.tensor as T

rng = np.random.RandomState(1234)

def unpickle(file):
    with open(file, 'rb') as f:
        data = cPickle.load(f)
    return data

trn = [unpickle('../cifar_10/data_batch_%d' %i) for i in range(1,6)]
cifar_X_1 = np.concatenate([d['data'] for d in trn]).astype('float32')
cifar_y_1 = np.concatenate([d['labels'] for d in trn]).astype('int32')

tst = unpickle('../cifar_10/test_batch')
cifar_X_2 = tst['data'].astype('float32')
cifar_y_2 = np.array(tst['labels'], dtype='int32')

cifar_X = np.r_[cifar_X_1, cifar_X_2]
cifar_y = np.r_[cifar_y_1, cifar_y_2]

cifar_X = cifar_X / 255.

train_X, test_X, train_y, test_y = train_test_split(cifar_X, cifar_y, test_size=0.2, random_state=??) # random_stateはひみつです
```

次のセルのhomework関数を完成させて提出してください

- **上記のコード以外で必要なもの**は全て書いてください

In [2]:
from __future__ import division
from collections import OrderedDict
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.cross_validation import train_test_split
from theano.tensor.nnet import conv2d
from theano.tensor.signal import pool
from theano.tensor.shared_randomstreams import RandomStreams

import cPickle
import numpy as np
import theano
import theano.tensor as T

rng = np.random.RandomState(1234)

def unpickle(file):
    with open(file, 'rb') as f:
        data = cPickle.load(f)
    return data


def load_cifar():
    trn = [unpickle('../cifar_10/data_batch_%d' %i) for i in range(1,6)]
    cifar_X_1 = np.concatenate([d['data'] for d in trn]).astype('float32')
    cifar_y_1 = np.concatenate([d['labels'] for d in trn]).astype('int32')

    tst = unpickle('../cifar_10/test_batch')
    cifar_X_2 = tst['data'].astype('float32')
    cifar_y_2 = np.array(tst['labels'], dtype='int32')

    cifar_X = np.r_[cifar_X_1, cifar_X_2]
    cifar_y = np.r_[cifar_y_1, cifar_y_2]

    cifar_X = cifar_X / 255.

    train_X, test_X, train_y, test_y = train_test_split(cifar_X, cifar_y, test_size=0.2, random_state=42)

    return (train_X, test_X, train_y, test_y)

def check_homework():
    train_X, test_X, train_y, test_y = load_cifar()
    pred_y = homework(train_X, test_X, train_y)
    return f1_score(test_y, pred_y, average='macro')

if 'homework' in globals():
    result = check_homework()
    print result
    
    print "No Error Occured!"

Using gpu device 0: GRID K520 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 4007)


Training cost: 0.810
EPOCH:: 1, Validation cost: 0.634, Validation F1: 0.773
time spent for one epoch 264.687087 sec
0.77160044973
No Error Occured!


In [1]:
def homework(train_X, test_X, train_y):
    import time
    
    
    # setup for cross-validation
    train_y = np.eye(10)[train_y].astype('int32')

    train_X = train_X.reshape((train_X.shape[0], 3, 32, 32))
    test_X = test_X.reshape((test_X.shape[0], 3, 32, 32))

    train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size=0.05)
    
    # data augmentation
    # flipping
    flip_train_X = train_X[:, :, :, ::-1]
    # cropping
    padded = np.pad(train_X, ((0, 0),(0, 0), (4, 4), (4, 4)), mode='constant')
    crops = np.random.randint(8, size=(len(train_X), 2))
    cropped_train_X = [padded[i, :, c[0]:(c[0]+32), c[1]:(c[1]+32)] for i, c in enumerate(crops)]
    cropped_train_X = np.array(cropped_train_X)
    
    # preprocessing
    # Global Contrast Normalization
    def gcn(x):
        mean = np.mean(x, axis=(1,2,3), keepdims=True)
        std = np.std(x, axis=(1,2,3), keepdims=True)
        return (x - mean)/std
    
    # ZCA whitening
    class ZCAWhitening:

        def __init__(self, epsilon=1e-4):
            self.epsilon = epsilon
            self.mean = None
            self.ZCA_matrix = None

        def fit(self, x):
            x = x.reshape(x.shape[0],-1)
            self.mean = np.mean(x,axis=0)
            x -= self.mean
            cov_matrix = np.dot(x.T, x) / x.shape[0] # expecation, thus divided by n
            A, d, _ = np.linalg.svd(cov_matrix)
            self.ZCA_matrix = np.dot(np.dot(A, np.diag(1.0 / np.sqrt(d + self.epsilon))), A.T)

        def transform(self, x):
            shape = x.shape
            x = x.reshape(x.shape[0], -1)
            x -= self.mean
            x = np.dot(x,self.ZCA_matrix.T)
            return x.reshape(shape)
        
    
    class BatchNorm:
        #- Constructor
        def __init__(self, shape, epsilon=np.float32(1e-5)):
            self.shape = shape
            self.epsilon = epsilon

            self.gamma = theano.shared(np.ones(self.shape, dtype="float32"), name="gamma")
            self.beta = theano.shared(np.zeros(self.shape, dtype="float32"), name="beta")
            self.params = [self.gamma, self.beta]

        #- Forward Propagation
        def f_prop(self, x):
            if x.ndim == 2:
                mean = T.mean(x, axis=0, keepdims=True)
                std = T.std(x, axis=0, keepdims=True)
            elif x.ndim == 4:
                mean = T.mean(x, axis=(0,2,3), keepdims=True) # (HINT : ndim=4のときはフィルタの次元でも平均をとる)
                std = T.sqrt(T.var(x, axis=(0,2,3), keepdims=True) + self.epsilon) # WRITE ME

            normalized_x = (x - mean) / std
            self.z = self.gamma * normalized_x + self.beta
            return self.z
        
    
    # CNN
    class Conv:
        #- Constructor
        def __init__(self, filter_shape, function=lambda x: x, border_mode="valid", subsample=(1, 1)):

            self.function = function
            self.border_mode = border_mode
            self.subsample = subsample

            fan_in = np.prod(filter_shape[1:])
            fan_out = (filter_shape[0] * np.prod(filter_shape[2:]))

            # Xavier
            self.W = theano.shared(rng.uniform(
                        low=-np.sqrt(6. / (fan_in + fan_out)),
                        high=np.sqrt(6. / (fan_in + fan_out)),
                        size=filter_shape
                    ).astype("float32"), name="W")
            self.b = theano.shared(np.zeros((filter_shape[0],), dtype="float32"), name="b")
            self.params = [self.W,self.b]

        #- Forward Propagation
        def f_prop(self, x):
            conv_out = conv2d(x, self.W, border_mode=self.border_mode, subsample=self.subsample)
            self.z = self.function(conv_out + self.b[np.newaxis, :, np.newaxis, np.newaxis])
            return self.z
        
    
    class Pooling:
        #- Constructor
        def __init__(self, pool_size=(2,2), padding=(0,0), mode='max'):
            self.pool_size = pool_size
            self.mode = mode
            self.padding = padding
            self.params = []

        #- Forward Propagation
        def f_prop(self, x):
            return pool.pool_2d(input=x, ds=self.pool_size, padding=self.padding, mode=self.mode, ignore_border=True)
        
        
    class Flatten:
        #- Constructor
        def __init__(self, outdim=2):
            self.outdim = outdim
            self.params = []

        #- Forward Propagation
        def f_prop(self,x):
            return T.flatten(x, self.outdim)
        
        
    class Layer:
        #- Constructor
        def __init__(self, in_dim, out_dim, function):
            self.in_dim = in_dim
            self.out_dim = out_dim
            self.function = function

            self.W = theano.shared(rng.uniform(
                        low=-np.sqrt(6. / (in_dim + out_dim)),
                        high=np.sqrt(6. / (in_dim + out_dim)),
                        size=(in_dim,out_dim)
                    ).astype("float32"), name="W")

            self.b =  theano.shared(np.zeros(out_dim).astype("float32"), name="b")
            self.params = [ self.W, self.b ]

        #- Forward Propagation
        def f_prop(self, x):
            self.z = self.function(T.dot(x, self.W) + self.b)
            return self.z
        
        
    class Activation:
        #- Constructor
        def __init__(self, function):
            self.function = function
            self.params = []

        #- Forward Propagation
        def f_prop(self, x):
            self.z = self.function(x)
            return self.z
        
        
    # Adam Optimizer
    def adam(cost, params, lr=0.001, b1=0.1, b2=0.001, e=1e-8):
        updates = []
        grads = T.grad(cost, params)
        i = theano.shared(np.asarray(0., dtype="float32"))
        i_t = i + 1.
        fix1 = 1. - (1. - b1)**i_t
        fix2 = 1. - (1. - b2)**i_t
        lr_t = lr * (T.sqrt(fix2) / fix1)
        for p, g in zip(params, grads):
            m = theano.shared(p.get_value() * 0.)
            v = theano.shared(p.get_value() * 0.)
            m_t = (b1 * g) + ((1. - b1) * m)
            v_t = (b2 * T.sqr(g)) + ((1. - b2) * v)
            g_t = m_t / (T.sqrt(v_t) + e)
            p_t = p - (lr_t * g_t)
            updates.append((m, m_t))
            updates.append((v, v_t))
            updates.append((p, p_t))
        updates.append((i, i_t))
        return updates
    
    
    # define network structure
    activation = T.nnet.relu

    layers = [                               # (チャネル数)x(縦の次元数)x(横の次元数)
        Conv((32, 3, 3, 3), border_mode=(1,1)),  # 3x32x32 -> 32x32x32
        BatchNorm((32, 32, 32)),
        Activation(activation),
        Conv((32, 32, 3, 3), border_mode=(1,1)), 
        BatchNorm((32, 32, 32)),
        Activation(activation),
        Conv((64, 32, 3, 3), border_mode=(1,1)), 
        BatchNorm((64, 32, 32)),
        Activation(activation),
        Pooling((2, 2)),                     #  64x32x32 ->  64x16x16
        Conv((64, 64, 3, 3), border_mode=(1,1)),                
        BatchNorm((64, 16, 16)),
        Activation(activation),
        Conv((64, 64, 3, 3), border_mode=(1,1)),                
        BatchNorm((64, 16, 16)),
        Activation(activation),
        Conv((128, 64, 3, 3), border_mode=(1,1)),                
        BatchNorm((128, 16, 16)),
        Activation(activation),
        Pooling((2, 2)),                     #  128x16x16 ->  128x 8x 8
        Conv((128, 128, 3, 3), border_mode=(1,1)),               
        BatchNorm((128, 8, 8)),
        Activation(activation),
        Conv((128, 128, 3, 3), border_mode=(1,1)),               
        BatchNorm((128, 8, 8)),
        Activation(activation),
        Conv((128, 128, 3, 3), border_mode=(1,1)),               
        BatchNorm((128, 8, 8)),
        Activation(activation),
        Pooling((2, 2)),                     # 128x 8x 8 -> 128x 4x 4
        Flatten(2),
        Layer(128*4*4, 512, activation),
        Layer(512, 10, T.nnet.softmax)
    ]
    
    
    x = T.ftensor4('x')
    t = T.imatrix('t')

    params = []
    layer_out = x
    for layer in layers:
        params += layer.params
        layer_out = layer.f_prop(layer_out)

    y = layers[-1].z

    cost = T.mean(T.nnet.categorical_crossentropy(y, t))

    updates = adam(cost, params, lr=0.0005, b1=0.1, b2=0.001, e=1e-4)

    train = theano.function(inputs=[x, t], outputs=cost, updates=updates, allow_input_downcast=True, name='train')
    valid = theano.function(inputs=[x, t], outputs=[cost, T.argmax(y, axis=1)], allow_input_downcast=True, name='valid')
    test  = theano.function(inputs=[x], outputs=T.argmax(y, axis=1), allow_input_downcast=True, name='test')
    
    # preprocessing data
    all_X = np.concatenate([train_X, flip_train_X, cropped_train_X])
    all_y = np.concatenate([train_y, train_y, train_y])
    zca = ZCAWhitening()
    zca.fit(gcn(all_X))
    zca_train_X = zca.transform(gcn(all_X))
    zca_train_y = all_y[:]
    zca_valid_X = zca.transform(gcn(valid_X))
    zca_valid_y = valid_y[:]
    zca_test_X  = zca.transform(gcn(test_X))
    
    batch_size = 100
    n_batches = zca_train_X.shape[0]//batch_size
    for epoch in xrange(8):
        tic = time.clock()
        zca_train_X, zca_train_y = shuffle(zca_train_X, zca_train_y)
        for i in xrange(n_batches):
            start = i*batch_size
            end = start + batch_size
            cost = train(zca_train_X[start:end], zca_train_y[start:end])
        print 'Training cost: %.3f' % cost
        valid_cost, pred_y = valid(zca_valid_X, zca_valid_y)
        print 'EPOCH:: %i, Validation cost: %.3f, Validation F1: %.3f' % (epoch + 1, valid_cost, 
                    f1_score(np.argmax(zca_valid_y, axis=1).astype('int32'), pred_y, average='macro'))
        print('time spent for one epoch %f sec' % (time.clock() - tic))
        
    # testing
    test_batch_size = 1000
    test_n_batches = zca_test_X.shape[0]//test_batch_size
    pred_ys= []
    for i in xrange(test_n_batches):
        start = i*test_batch_size
        end = start + test_batch_size
        pred_ys.append(test(zca_test_X[start:end]))

    pred_y = np.concatenate(pred_ys)
    
    return pred_y

In [ ]:
from __future__ import division
from collections import OrderedDict
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.cross_validation import train_test_split
from theano.tensor.nnet import conv2d
from theano.tensor.signal import pool
from theano.tensor.shared_randomstreams import RandomStreams

import cPickle
import numpy as np
import theano
import theano.tensor as T

rng = np.random.RandomState(1234)

def unpickle(file):
    with open(file, 'rb') as f:
        data = cPickle.load(f)
    return data


def load_cifar():
    trn = [unpickle('../cifar_10/data_batch_%d' %i) for i in range(1,6)]
    cifar_X_1 = np.concatenate([d['data'] for d in trn]).astype('float32')
    cifar_y_1 = np.concatenate([d['labels'] for d in trn]).astype('int32')

    tst = unpickle('../cifar_10/test_batch')
    cifar_X_2 = tst['data'].astype('float32')
    cifar_y_2 = np.array(tst['labels'], dtype='int32')

    cifar_X = np.r_[cifar_X_1, cifar_X_2]
    cifar_y = np.r_[cifar_y_1, cifar_y_2]

    cifar_X = cifar_X / 255.

    train_X, test_X, train_y, test_y = train_test_split(cifar_X, cifar_y, test_size=0.2, random_state=42)

    return (train_X, test_X, train_y, test_y)

def check_homework():
    train_X, test_X, train_y, test_y = load_cifar()
    pred_y = homework(train_X, test_X, train_y)
    return f1_score(test_y, pred_y, average='macro')

if 'homework' in globals():
    result = check_homework()
    
    print "No Error Occured!"